# Attention
Analyze distribution of attention values.  
The average mean temperature of a nucleus is 0.64.   
The median mean temperature of a nucleus is 0.71.
If the patch-wide mean is 0.50, then nuclei get more attention on average.

In [1]:
import numpy as np
import pandas as pd
import os
from datetime import datetime
print(datetime.now())

2022-08-11 06:17:29.948207


In [2]:
BASE_PATH='/home/jrm/Adjeroh/Glioma/August_Run/CellProfilerPerPatient/'
FOLDS_META_FILE = 'D:\\Adjeroh\\Glioma\\August_Run\\TrainValidSplits\\aug_train_valid_splits.csv'  # Windows
FOLDS_META_FILE = '/home/jrm/Adjeroh/Glioma/August_Run/TrainValidSplits/aug_train_valid_splits.csv'
# Cross validation
NUM_FOLDS = 5
# Patch filename format: TCGA-06-0129-01Z-00-DX1_5400_5100.png
# For WSI ID, use first 23 letters.
# For patient or case ID, use first 19 letters. 
LEN_CASE_ID=19
LEN_WSI_ID=23

In [3]:
METADATA = pd.read_csv(FOLDS_META_FILE)
def get_metadata_one_fold(fold,is_train):
    df = METADATA.loc[METADATA['Fold']==fold]
    if is_train:
        df = df.loc[df['Split']=='Train']
    else:
        df = df.loc[df['Split']=='Valid']
    patients = df['Case'].tolist()
    labels = df['Class'].tolist()
    return patients,labels

In [4]:
def get_patient_attention(fold,is_train,patient):
    cls = 'mcls'
    filename = f"Attention_{cls}_fold{fold}.csv"
    filepath = BASE_PATH + patient + '/' + filename
    df = pd.read_csv(filepath)
    return df

In [5]:
# This is a sample 80% of the data
fold = 0
train = True

patients,cancers = get_metadata_one_fold(fold,train)
att_df = None
for patient in patients:
    df = get_patient_attention(fold,train,patient)
    if len(df)<1:
        raise Exception ('No data for patient '+patient)
    if train:
        if df.iloc[0]['Usage']!='train' or df['Usage'].nunique()!=1:
            raise Exeption ('Expected training data for patient '+patient)
    else:
        if df.iloc[0]['Usage']!='valid' or df['Usage'].nunique()!=1:
            raise Exeption ('Expected validation data for patient '+patient)
    bad_cols = ['ImageName','ImageNumber','ObjectNumber','Usage']
    df = df.drop(columns=bad_cols)
    if att_df is None:
        att_df = df
    else:
        att_df = pd.concat( (att_df,df) )

In [8]:
quintiles = [x/5 for x in range(6)]
att_df.describe(percentiles=quintiles)

,MedTemp,AvgTemp,MaxTemp
count,1.482799e+06,1.482799e+06,1.482799e+06
mean,6.404298e-01,6.408631e-01,7.084842e-01
std,2.388129e-01,2.362207e-01,2.244737e-01
min,0.000000e+00,0.000000e+00,0.000000e+00
0%,0.000000e+00,0.000000e+00,0.000000e+00
20%,3.995511e-01,4.045955e-01,4.942462e-01
40%,6.284506e-01,6.282173e-01,7.209109e-01
50%,7.156081e-01,7.144382e-01,7.837816e-01
60%,7.731838e-01,7.715611e-01,8.287602e-01
80%,8.567473e-01,8.548153e-01,8.995662e-01


In [7]:
print(datetime.now())

2022-08-11 06:17:32.164648
